# Régression

## Objectif

Dans cette partie "Régression", on cherche à prédire les émissions de CO2 de voitures, en se basant sur leurs caractéristiques \(poids, carburant utilisé, etc\). L'objectif est de minimiser la moyenne des erreurs absolues entre les valeurs prédites et les valeurs réelles.

## Traitement des données

### Importation des dépendances

Nous importons les dépendances python en début de fichier pour faciliter leur installation.

# Regréssion avec Scikit learn

Dans cette partie, nous allons reprendre notre reégrression linéaire mais avec la bibliothèque Scikit-learn, qui propose des modèles très interessants

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd
import sklearn

## Importation des données

In [2]:
df_train = pd.read_csv("dataset/train.csv")
df_test = pd.read_csv("dataset/test.csv")

On supprime la colonne "id". On sépare la cible des autres variables

In [3]:
target_name = "co2"
target, data = df_train[target_name] , df_train.drop(columns=[target_name,"id"])   #On sépare la colonne target des autres colonnes

## Vérifions les valeurs nulles

Beaucoup de valeurs nulles dans les colonnes hc, nox, hcnox

In [61]:
data.isna().sum()

brand                   0
model                   0
car_class               0
range                   0
fuel_type               0
hybrid                  0
max_power               0
grbx_type_ratios        0
weight_min              0
weight_max              0
urb_cons                6
exturb_cons             6
overall_cons            0
co                     86
hc                  33890
nox                    86
hcnox                7416
ptcl                 1965
dtype: int64

## Parcours des données

In [62]:
data.head()

,brand,model,car_class,range,fuel_type,hybrid,max_power,grbx_type_ratios,weight_min,weight_max,urb_cons,exturb_cons,overall_cons,co,hc,nox,hcnox,ptcl
0,MERCEDES,COMBI 110 CDI,MINIBUS,MOY-INFER,GO,non,70.0,M 6,1976,2075,9.1,6.4,7.4,0.083,NaN,0.229,0.250,0.001
1,MERCEDES,VIANO 2.0 CDI,MINIBUS,MOY-SUPER,GO,non,100.0,A 5,2186,2355,10.2,7.0,8.2,0.078,NaN,0.224,0.233,0.001
2,MERCEDES,SPRINTER COMBI 319 CDI,MINIBUS,MOY-INFER,GO,non,140.0,A 5,2586,2869,12.5,9.0,10.3,0.067,0.014,1.846,NaN,0.002
3,RENAULT,MEGANE Coupé EnergyTCe (115ch) eco2,COUPE,MOY-INFER,ES,non,85.0,M 6,1280,1280,6.4,4.6,5.3,0.167,0.039,0.039,NaN,0.001
4,MERCEDES,COMBI 116 CDI,MINIBUS,MOY-INFER,GO,non,120.0,A 5,2356,2450,10.1,6.9,8.1,0.042,NaN,0.190,0.201,0.001


In [63]:
data.describe()

,max_power,weight_min,weight_max,urb_cons,exturb_cons,overall_cons,co,hc,nox,hcnox,ptcl
count,41257.000000,41257.000000,41257.000000,41251.000000,41251.000000,41257.000000,41171.000000,7367.000000,41171.000000,33841.000000,39292.000000
mean,118.982960,2102.832901,2342.408609,9.570216,6.728448,7.761779,0.181025,0.026455,0.300336,0.233684,0.000896
std,45.282568,293.810948,423.303792,2.079990,1.035866,1.378721,0.145250,0.019502,0.418976,0.037348,0.000994
min,28.000000,825.000000,825.000000,0.000000,2.800000,0.600000,0.005000,0.000000,0.000000,0.000000,0.000000
25%,100.000000,1982.000000,2075.000000,8.800000,6.500000,7.300000,0.061000,0.008000,0.197000,0.216000,0.000000
50%,120.000000,2076.000000,2355.000000,9.400000,6.900000,7.800000,0.137000,0.029000,0.214000,0.239000,0.001000
75%,120.000000,2246.000000,2709.000000,10.200000,7.200000,8.300000,0.297000,0.042000,0.228000,0.253000,0.001000
max,585.000000,2760.000000,3094.000000,41.099998,14.900000,24.500000,0.968000,0.510000,1.846000,0.570000,0.023000


## Transformation des Données

----- D'après la description des données hcnox = hc +nox

Ainsi, on peut donc retrouver les valeurs nulles dans la colonne hc à partir de hcnox et nox. Une fois cela fait, on supprime la colonne hcnox.

In [4]:
# Vérifier si la colonne "hcnox" est vide
is_empty = (data['hc'].isnull() & data['hcnox'].notnull() & data['nox'].notnull())

# Remplir les valeurs manquantes dans "hcnox" avec la somme de "hc" et "nox"
data.loc[is_empty,'hc'] = data.loc[is_empty,'hcnox'] - data.loc[is_empty,'nox']
data = data.drop(columns=["hcnox"])

---- Variable Range

In [5]:
data["range"].value_counts()

range
MOY-INFER         25118
MOY-SUPER         11275
LUXE               2677
SUPERIEURE         1163
INFERIEURE          800
ECONOMIQUE          186
MOY-INFERIEURE       38
Name: count, dtype: int64

### --- Model
Trop de valeurs différentes. Il serait judicieux de la supprimer

In [131]:
print(data["model"].value_counts())
print(data["model"].nunique())

model
VIANO 2.2 CDI                                                  4405
VIANO 2.0 CDI                                                  2931
COMBI 116 CDI                                                  2847
COMBI 113 CDI                                                  1968
VIANO 3.0 CDI                                                  1180
                                                               ... 
2008 1.6 e-Hdi FAP BVM6                                           1
A6 2.0 TDI (136ch) MULTITRONIC 8                                  1
PASSAT SW 1.6 TDI (105ch) CR FAP BMT BVM6                         1
MULTIVAN CONFORTLINE ET HIGHLINE 2.0 TDI (180ch) BlueMotion       1
ASX CLEARTEC 1.8 DI-D Invite / Intense / Instyle 4WD              1
Name: count, Length: 3141, dtype: int64
3141


In [ ]:
data = data.drop(columns="model")

### --- Car Class

Rien à faire pour cette variable à priori

In [133]:
print(data["car_class"].value_counts())
print(data["car_class"].nunique())

car_class
MINIBUS                34570
BERLINE                 3307
BREAK                    933
TS TERRAINS/CHEMINS      819
COUPE                    661
CABRIOLET                389
MONOSPACE COMPACT        240
COMBISPACE               179
MINISPACE                111
MONOSPACE                 44
COMBISPCACE                4
Name: count, dtype: int64
11


### ---- Variable grbx_type_ratios

Scinder la colonnne "grbx_type_ratios" en deux colonnes :  le Gear_box type et le nombre de ratio

In [66]:
data["grbx_type_ratios"].value_counts()

grbx_type_ratios
M 6    23823
A 5    10169
A 7     5103
M 5      738
A 6      647
A 8      385
V 0      238
D 5       51
A 9       27
D 6       23
A 4       19
D 7       18
M 7       13
V .        2
S 6        1
Name: count, dtype: int64

In [115]:
data[['gearbox', 'ratio']] = data["grbx_type_ratios"].str.split(' ', n=1, expand=True)
data['ratio'] = data['ratio'].replace({'.': '0'})                                          #Remplacer les points par zéro
data['ratio'] = pd.to_numeric(data['ratio'], errors='coerce').fillna(0)
data = data.drop(columns="grbx_type_ratios")

----- Variable Brand ??

Beaucoup de valeurs différentes (44). On peut conserver les deux premières car elles sont les plus présentes dans le dataset

In [68]:
print(data["brand"].value_counts())
print(f"Nombre de valeurs différentes dans la colonne Brand {data['brand'].nunique()}")

brand
MERCEDES        27127
VOLKSWAGEN      10266
FIAT              394
BMW               342
OPEL              308
LEXUS             277
AUDI              264
FORD              220
NISSAN            215
CITROEN           166
ALFA-ROMEO        142
PEUGEOT           125
RENAULT           109
TOYOTA            103
SKODA              96
MINI               95
VOLVO              86
PORSCHE            82
SEAT               78
CHEVROLET          61
JEEP               60
LAND ROVER         60
ASTON MARTIN       55
KIA                53
HONDA              43
HYUNDAI            43
LANCIA             41
MITSUBISHI         35
MAZDA              31
CADILLAC           31
JAGUAR             29
SMART              29
LAMBORGHINI        26
DACIA              22
SUZUKI             21
SUBARU             19
LOTUS              18
FERRARI            18
MASERATI           16
ROLLS-ROYCE        15
BENTLEY            14
INFINITI           12
SSANGYONG           6
LADA                4
Name: count, dtype: int64


In [116]:
is_not_abundant = ~data["brand"].isin(["MERCEDES", "VOLKSWAGEN"])
data.loc[is_not_abundant, "brand"] = "AUTRE"

### ---- Fuel Type

Nous avons d'abord voulu fusionner GN/ES et ES/GN. Mais après des recherches, nous avons vu que c'est deux etiquetes differentes

In [6]:
print(data["fuel_type"].value_counts())
print(data["fuel_type"].nunique())

fuel_type
GO       36993
ES        3853
EH         233
GH          63
GN/ES       27
ES/GN       23
ES/GP       18
GP/ES       17
GN          15
FE           8
EE           6
GL           1
Name: count, dtype: int64
12


In [70]:
print(data["brand"].value_counts())

brand
MERCEDES      27127
VOLKSWAGEN    10266
AUTRE          3864
Name: count, dtype: int64


### Utilisons Scikit learn pour diviser nos données

In [77]:
x_train, x_val, y_train, y_val = train_test_split(data, target, test_size=0.2, random_state=1)
x_train = x_train.reset_index().drop('index', axis=1)
x_val = x_val.reset_index().drop('index', axis=1)

In [78]:
x_train.head()

,brand,model,car_class,range,fuel_type,hybrid,max_power,weight_min,weight_max,urb_cons,exturb_cons,overall_cons,co,hc,nox,ptcl,gearbox,ratio
0,MERCEDES,SPRINTER COMBI 213 CDI - 37,MINIBUS,MOY-INFER,GO,non,95.0,2076,2185,10.0,6.6,7.9,0.483,0.066,0.221,0.000,M,6
1,MERCEDES,SPRINTER COMBI 213 CDI - 37,MINIBUS,MOY-INFER,GO,non,95.0,2356,2585,10.1,7.4,8.4,0.100,0.006,0.247,0.001,A,7
2,VOLKSWAGEN,CRAFTER COMBI 35 L4H3 TDI (163ch) BlueMotion,MINIBUS,MOY-INFER,GO,non,120.0,2307,2815,8.0,6.6,7.1,0.401,0.038,0.213,0.001,M,6
3,MERCEDES,VIANO 2.0 CDI,MINIBUS,MOY-SUPER,GO,non,100.0,2075,2075,8.8,6.4,7.2,0.175,0.022,0.226,0.001,M,6
4,MERCEDES,COMBI 116 CDI,MINIBUS,MOY-INFER,GO,non,120.0,2186,2355,8.9,6.5,7.3,0.046,0.019,0.197,0.000,M,6


## Sélection des variables numériques et catégoriques

In [117]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

## Pipeline de préprocessing

In [118]:
categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Define the preprocessing steps for numeric variables
numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])


#Notre modèle qui est un pipeline, standarisation des données puis, regréssion logistique
preprocessors = ColumnTransformer([
    ('categorical preprocessing', categorical_preprocessor, categorical_columns),
    ('numerical preprocessing', numerical_preprocessor, numerical_columns)])

## Construction des modèles

Faisons d'abord les fonctions de prédictions et de Cross Validation

### Fonction pour le prétraitement des données de test

In [119]:
def preprocess(df_test):
    is_empty = (df_test['hc'].isnull() & df_test['hcnox'].notnull() & df_test['nox'].notnull())

    # Remplir les valeurs manquantes dans "hcnox" avec la somme de "hc" et "nox"
    df_test.loc[is_empty,'hc'] = df_test.loc[is_empty,'hcnox'] - df_test.loc[is_empty,'nox']
    df_test = df_test.drop(columns=["id","hcnox","model"])

    df_test[['gearbox', 'ratio']] = df_test["grbx_type_ratios"].str.split(' ', n=1, expand=True)
    df_test['ratio'] = df_test['ratio'].replace({'.': '0'})                                          #Remplacer les points par zéro
    df_test['ratio'] = pd.to_numeric(df_test['ratio'], errors='coerce').fillna(0)
    df_test = df_test.drop(columns="grbx_type_ratios")

    is_not_abundant = ~df_test["brand"].isin(["MERCEDES", "VOLKSWAGEN"])
    df_test.loc[is_not_abundant, "brand"] = "AUTRE"

    return df_test

### Fonction pour avoir les prédictions

In [91]:
def get_prediction(X_val, y_val, model):
    pred = model.predict(X_val) # Your code here
    
    # Calculate MAE
    mae = mean_absolute_error(y_val, pred)

    # Uncomment to print MAE
    print("Mean Absolute Error:" , mae)

### Fonction pour la Cross Validation

In [92]:
def validation(model,nb_validate):    
    cv_results = cross_validate(
        model, data, target, cv=nb_validate, return_estimator=True,           #nb_validate validations croisées
        return_train_score=True, scoring="neg_mean_squared_error"
    )
    scores_train=-cv_results["train_score"]
    scores_test=-cv_results["test_score"]
    print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:\n"
        f"{scores_train.mean():.3f} ± {scores_train.std():.3f}")

    print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:\n"
        f"{scores_test.mean():.3f} ± {scores_test.std():.3f}")    

## -- XGBoost Pipeline

In [124]:
xg_regressor = XGBRegressor(n_estimators=10000, 
                     random_state=0, 
                     learning_rate=0.01,
                     max_depth=100,
                     enable_categorical=True)

xg_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessors),
        ('regressor', xg_regressor)
    ])

In [ ]:
xg_regressor.fit(x_train,y_train)

### Hyper parameter Tuning du XGBoost

In [ ]:
param_grid = {
    'model__n_estimators': [1500,2000, 2300, 2500,3000],
    'model__max_depth': [3, 6, 9, 12],
    'model__learning_rate': [0.07, 0.1, 0.13, 0.15, 0.2]
}

RandomizedSearchCV a été préféré à GridSearchCV car GridSearchCV teste toutes les combinaisons ce qui prend énormément de temps.

In [ ]:
random_cv = RandomizedSearchCV(
    estimator=xg_pipeline,
    param_distributions=param_grid,
    cv=3,                                           #Trois validations croisées pour déterminer les meilleures hyper paramètres
    n_iter=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=5,
    return_train_score=True,
    return_test_score=True,
    random_state=42
)

In [ ]:
random_cv.fit(X_train, y_train)

In [ ]:
random_cv.best_params_

### Testons notre modèle et création du fichier pour le submit sur Kaggle

In [125]:
xg_pipeline.fit(data,target)

KeyboardInterrupt: 

In [122]:
df_test_processed = preprocess(df_test)
predictions  = xg_pipeline.predict(df_test_processed)

In [123]:
submit = pd.DataFrame({'id': df_test["id"], 'co2': predictions})
submit.to_csv("predictions.csv", index=False)

## -- AdaBoost

In [ ]:
ada = make_pipeline(preprocessors, AdaBoostRegressor(n_estimators=4000, learning_rate=1.0, loss='linear'))
ada_regressor = ada.fit(data,target)

## -- RidgeCV

In [147]:
alphas = np.logspace(-5, 5, num=101)
ridge = make_pipeline(preprocessors, RidgeCV(alphas=alphas,cv=2))
ridge_regressor = ridge.fit(data,target)

Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:
0.225 ± 0.010
Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:
0.282 ± 0.040


## Régression linéaire

In [112]:
from sklearn.linear_model import LinearRegression
model2 = make_pipeline(preprocessors, LinearRegression())
cv_results2 = cross_validate(
    model2, data, target, cv=2, return_estimator=True,
    return_train_score=True, scoring="neg_mean_squared_error"
)
scores_train2=-cv_results2["train_score"]
scores_test2=-cv_results2["test_score"]

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:\n"
      f"{scores_train2.mean():.3f} ± {scores_train2.std():.3f}")

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:\n"
      f"{scores_test2.mean():.3f} ± {scores_test2.std():.3f}")
#ExtraTreesRegressor(n_estimators=10, max_features=32, random_state=0),

Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:
0.206 ± 0.001
Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:
0.628 ± 0.089


## K Nearest Neighbors

In [145]:
model3 = make_pipeline(preprocessors, KNeighborsRegressor(n_neighbors=5,weights="distance",p=1))
cv_results3 = cross_validate(
    model3, data, target, cv=2, return_estimator=True,
    return_train_score=True, scoring="neg_mean_squared_error"
)

scores_train3=-cv_results3["train_score"]
scores_test3=-cv_results3["test_score"]

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:\n"
      f"{scores_train3.mean():.3f} ± {scores_train3.std():.3f}")

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:\n"
      f"{scores_test3.mean():.3f} ± {scores_test3.std():.3f}")

Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:
0.005 ± 0.000
Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:
1.760 ± 0.431


In [100]:
from sklearn.ensemble import GradientBoostingRegressor
model5 = make_pipeline(preprocessors, GradientBoostingRegressor(n_estimators=1000, max_features=20))
cv_results5 = cross_validate(
    model5, data, target, cv=2, return_estimator=True,
    return_train_score=True, scoring="neg_mean_squared_error"
)

scores_train5=-cv_results5["train_score"]
scores_test5=-cv_results5["test_score"]

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:\n"
      f"{scores_train5.mean():.3f} ± {scores_train5.std():.3f}")

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:\n"
      f"{scores_test5.mean():.3f} ± {scores_test5.std():.3f}")


Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:
4.937 ± 0.449
Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:
10.236 ± 1.854


In [117]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
 
#reg = StackingRegressor(estimators=RidgeCV(alphas=alphas,cv=2),final_estimator=KNeighborsRegressor(n_neighbors=20,weights="distance",p=1))
r1 = RidgeCV(alphas=alphas,cv=2)
r2 = KNeighborsRegressor(n_neighbors=20,weights="distance",p=1)
model4 = make_pipeline(preprocessors,VotingRegressor(estimators=[('lr',r1),('rf',r2)]))

In [118]:
cv_results4 = cross_validate(
    model4, data, target, cv=2, return_estimator=True,
    return_train_score=True, scoring="neg_mean_squared_error"
)

scores_train4=-cv_results4["train_score"]
scores_test4=-cv_results4["test_score"]

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:\n"
      f"{scores_train4.mean():.3f} ± {scores_train4.std():.3f}")

print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:\n"
      f"{scores_test4.mean():.3f} ± {scores_test4.std():.3f}")

Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:
0.064 ± 0.000
Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:
0.896 ± 0.103
